In [2]:
# type your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor,StackingRegressor
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score,precision_score
from sklearn.metrics import mean_squared_log_error

In [3]:
train = pd.read_csv('TRAIN.csv')
test = pd.read_csv('TEST_FINAL.csv')

In [4]:
train.head(10)

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52
5,T1000006,249,S1,L3,R2,2018-01-01,1,Yes,39,34211.22
6,T1000007,248,S1,L1,R2,2018-01-01,1,Yes,40,35352.66
7,T1000008,247,S1,L1,R3,2018-01-01,1,Yes,64,52650.00
8,T1000009,246,S3,L1,R3,2018-01-01,1,Yes,62,42633.78
9,T1000010,254,S4,L1,R1,2018-01-01,1,Yes,87,62572.80


In [5]:
train.describe()

,Store_id,Holiday,#Order,Sales
count,188340.000000,188340.000000,188340.000000,188340.000000
mean,183.000000,0.131783,68.205692,42784.327982
std,105.366308,0.338256,30.467415,18456.708302
min,1.000000,0.000000,0.000000,0.000000
25%,92.000000,0.000000,48.000000,30426.000000
50%,183.000000,0.000000,63.000000,39678.000000
75%,274.000000,0.000000,82.000000,51909.000000
max,365.000000,1.000000,371.000000,247215.000000


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188340 entries, 0 to 188339
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             188340 non-null  object 
 1   Store_id       188340 non-null  int64  
 2   Store_Type     188340 non-null  object 
 3   Location_Type  188340 non-null  object 
 4   Region_Code    188340 non-null  object 
 5   Date           188340 non-null  object 
 6   Holiday        188340 non-null  int64  
 7   Discount       188340 non-null  object 
 8   #Order         188340 non-null  int64  
 9   Sales          188340 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 14.4+ MB


In [6]:
train_obj = train.select_dtypes(include='object')
train_num = train.select_dtypes(include='number')

In [26]:
for i in train_obj.columns:
    print(i)
    print(train_obj[i].value_counts())
    print()
    print()

ID
T1102722    1
T1076199    1
T1083925    1
T1080431    1
T1003668    1
           ..
T1130983    1
T1110169    1
T1115088    1
T1018469    1
T1016908    1
Name: ID, Length: 188340, dtype: int64


Store_Type
S1    88752
S4    45924
S2    28896
S3    24768
Name: Store_Type, dtype: int64


Location_Type
L1    85140
L2    48504
L3    29928
L5    13932
L4    10836
Name: Location_Type, dtype: int64


Region_Code
R1    63984
R2    54180
R3    44376
R4    25800
Name: Region_Code, dtype: int64


Date
2018-05-22    365
2018-03-18    365
2019-01-22    365
2018-01-22    365
2018-03-16    365
             ... 
2018-06-06    365
2018-04-16    365
2019-01-06    365
2018-12-23    365
2018-07-15    365
Name: Date, Length: 516, dtype: int64


Discount
No     104051
Yes     84289
Name: Discount, dtype: int64




In [27]:
for i in train_num.columns:
    print(i)
    print(train_num[i].value_counts())
    print()
    print()

Store_id
255    516
38     516
294    516
39     516
167    516
      ... 
337    516
82     516
210    516
338    516
128    516
Name: Store_id, Length: 365, dtype: int64


Holiday
0    163520
1     24820
Name: Holiday, dtype: int64


Sales
31362.00     32
42033.00     31
39354.00     29
37962.00     28
32010.00     28
             ..
27516.75      1
28278.72      1
30309.75      1
115140.00     1
18032.58      1
Name: Sales, Length: 47422, dtype: int64




In [7]:
train = train.drop(['ID','Date'],axis=1)

In [8]:
train.head()

,Store_id,Store_Type,Location_Type,Region_Code,Holiday,Discount,#Order,Sales
0,1,S1,L3,R1,1,Yes,9,7011.84
1,253,S4,L2,R1,1,Yes,60,51789.12
2,252,S3,L2,R1,1,Yes,42,36868.20
3,251,S2,L3,R1,1,Yes,23,19715.16
4,250,S2,L3,R4,1,Yes,62,45614.52


In [9]:
train['Holiday'] = train['Holiday'].astype('object')

In [10]:
dumm = {'No':0,'Yes':1}
train['Discount'] = train['Discount'].map(dumm)

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188340 entries, 0 to 188339
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Store_id       188340 non-null  int64  
 1   Store_Type     188340 non-null  object 
 2   Location_Type  188340 non-null  object 
 3   Region_Code    188340 non-null  object 
 4   Holiday        188340 non-null  object 
 5   Discount       188340 non-null  int64  
 6   #Order         188340 non-null  int64  
 7   Sales          188340 non-null  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 11.5+ MB


In [12]:
train['Discount'] = train['Discount'].astype('object')

In [13]:
print('before',train.shape)
train = pd.get_dummies(train,drop_first=True)
print('after shape',train.shape)

before (188340, 8)
after shape (188340, 15)


In [14]:
train.head()

,Store_id,#Order,Sales,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R2,Region_Code_R3,Region_Code_R4,Holiday_1,Discount_1
0,1,9,7011.84,0,0,0,0,1,0,0,0,0,0,1,1
1,253,60,51789.12,0,0,1,1,0,0,0,0,0,0,1,1
2,252,42,36868.20,0,1,0,1,0,0,0,0,0,0,1,1
3,251,23,19715.16,1,0,0,0,1,0,0,0,0,0,1,1
4,250,62,45614.52,1,0,0,0,1,0,0,0,0,1,1,1


In [15]:
X = train.drop('Sales',axis=1)
y = train.Sales
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [16]:
test_d = test.drop(['ID','Date'],axis=1)

In [17]:
test_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22265 entries, 0 to 22264
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Store_id       22265 non-null  int64 
 1   Store_Type     22265 non-null  object
 2   Location_Type  22265 non-null  object
 3   Region_Code    22265 non-null  object
 4   Holiday        22265 non-null  int64 
 5   Discount       22265 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.0+ MB


In [18]:
test_d['Holiday'] = test_d['Holiday'].astype('object')
dumm = {'No':0,'Yes':1}
test_d['Discount'] = test_d['Discount'].map(dumm)
test_d['Discount'] = test_d['Discount'].astype('object')

In [19]:
print('before',test_d.shape)
test_d = pd.get_dummies(test_d,drop_first=True)
print('after shape',test_d.shape)

before (22265, 6)
after shape (22265, 13)


In [20]:
ID = test['ID']

In [21]:
rfc = RandomForestRegressor()
rfc_model_whole = rfc.fit(X_train,y_train)
y_pred = rfc_model_whole.predict(X_test)
mean_squared_log_error(y_test,y_pred)

0.009394404793321601

In [76]:
y_pred_test = rfc_model_whole.predict(test_d)
y_pred_series = pd.Series(y_pred_test)

data = {"ID": ID,
        "Sales": y_pred_series}
df = pd.concat(data,
               axis = 1)
new_df = df.set_index("ID")
submission_data = new_df.to_csv('final submission_3_rf.csv')

In [78]:
Ada = AdaBoostRegressor()
ada_model_whole = Ada.fit(X_train,y_train)
y_pred = ada_model_whole.predict(X_test)
mean_squared_log_error(y_test,y_pred)

0.14535002359768037

In [79]:
Grd = GradientBoostingRegressor()
grd_model_whole = Grd.fit(X_train,y_train)
y_pred = grd_model_whole.predict(X_test)
mean_squared_log_error(y_test,y_pred)

0.10499764376013299

In [22]:
from xgboost import XGBRegressor

In [23]:
xgrd = XGBRegressor()
xgrd_model_whole = xgrd.fit(X_train,y_train)
y_pred = xgrd_model_whole.predict(X_test)
mean_squared_log_error(y_test,y_pred)

0.008939968583578822

In [82]:
y_pred_test = xgrd_model_whole.predict(test_d)
y_pred_series = pd.Series(y_pred_test)

data = {"ID": ID,
        "Sales": y_pred_series}
df = pd.concat(data,
               axis = 1)
new_df = df.set_index("ID")
submission_data = new_df.to_csv('submission_4_xg.csv')

In [90]:
tuning_parameters = {'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
                     'max_depth': range(3,10),
                     'gamma': [0, 1, 2, 3, 4]}

# instantiate the 'XGBClassifier' 
xgb_model = XGBRegressor()

# use GridSearchCV() to find the optimal value of the hyperparameters
# estimator: pass the XGBoost classifier model
# param_grid: pass the list 'tuned_parameters'
# cv: number of folds in k-fold i.e. here cv = 3
# scoring: pass a measure to evaluate the model on test set
xgb_grid = GridSearchCV(estimator = xgb_model, param_grid = tuning_parameters, cv = 3, scoring = 'roc_auc')

# fit the model on X_train and y_train using fit()
xgb_grid.fit(X_train, y_train)

# get the best parameters
print('Best parameters for XGBoost classifier: ', xgb_grid.best_params_, '\n')

C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: continuous format is not supported

  warnings.warn(
C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to n

C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: continuous format is not supported

  warnings.warn(
C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to n

C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: continuous format is not supported

  warnings.warn(
C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to n

C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: continuous format is not supported

  warnings.warn(
C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to n

C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: continuous format is not supported

  warnings.warn(
C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to n

...

C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: continuous format is not supported

  warnings.warn(
C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to n

C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: continuous format is not supported

  warnings.warn(
C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to n

C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: continuous format is not supported

  warnings.warn(
C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to n

C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\aniru\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: continuous format is not supported

  warnings.warn(
C:\Users\aniru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to n

Best parameters for XGBoost classifier:  {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3} 



In [92]:
xgb_grid_model = XGBRegressor(learning_rate = xgb_grid.best_params_.get('learning_rate'),
                               max_depth = xgb_grid.best_params_.get('max_depth'),
                              gamma = xgb_grid.best_params_.get('gamma'))

# use fit() to fit the model on the train set
xgb_model = xgb_grid_model.fit(X_train, y_train)

In [93]:
y_pred = xgb_model.predict(X_test)
mean_squared_log_error(y_test,y_pred)

0.10497745496343648

In [94]:
y_pred_test = xgb_model.predict(test_d)
y_pred_series = pd.Series(y_pred_test)

data = {"ID": ID,
        "Sales": y_pred_series}
df = pd.concat(data,
               axis = 1)
new_df = df.set_index("ID")
submission_data = new_df.to_csv('submission_5_xgboost_original.csv')

In [96]:
base_learners = [('rf_model', RandomForestRegressor()),
                 ('KNN_model', XGBRegressor())]

stack_model = StackingRegressor(estimators = base_learners, final_estimator = XGBRegressor())

# fit the model on train dataset
stack_model.fit(X_train, y_train)

StackingRegressor(estimators=[('rf_model', RandomForestRegressor()),
                              ('KNN_model',
                               XGBRegressor(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=na...
                                               importance_type='gain',
                                               interaction_constraints=None,
                                               learning_rate=None,
  

In [97]:
y_pred = stack_model.predict(X_test)
mean_squared_log_error(y_test,y_pred)

0.09910126411094082

In [98]:
y_pred_test = stack_model.predict(test_d)
y_pred_series = pd.Series(y_pred_test)

data = {"ID": ID,
        "Sales": y_pred_series}
df = pd.concat(data,
               axis = 1)
new_df = df.set_index("ID")
submission_data = new_df.to_csv('submission_6_stack.csv')